In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling/")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Math stuff
import numpy as np

# Data visualisation
import pandas as pd

# Vectorize algotrithm
from gensim.models import Word2Vec

# Support Vector Classifier Algorithm
from sklearn.svm import SVC
# :gamma: is a parameter for non linear hyperplanes. The higher the gamma
#         value it tries to exactly fit the training data set.
GAMMA = 30
# :C: is the penalty parameter of the error term. It controls the trade
#     off between smooth decision boundary and classifying the training points correctly.
#     !!! Increasing C values may lead to overfitting the training data. !!!
C = 100


# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split
RANDOM_STATE = 42
# Where "train" contains 70% of data and "test" - 30%
TEST_SIZE = 0.3
SHUFFLE = True


# Mean embedding function
from mean_vectorizer import MeanEmbeddingVectorizer

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

# Transform DataFrame to dictionary
from dictionary_transformation import series_pair_to_dict


In [ ]:
# Read data
df = pd.read_pickle(TABLE_PATH)
df.head()

In [3]:
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
category = df['Category']

## word2vec model

In [4]:
# Insert our data and set minimal word count to 10, and size of each word to 300 vectors
model = Word2Vec.load('/home/jovyan/work/Dan/data/pwdb/word2vec/df.model')
w2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [5]:
# Use pipes to implement steps of fit and transform method
svm_w2v = Pipeline([
    # Add the words we want to mean
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v_dict)),
    ("SVM", SVC(gamma=GAMMA, C=C))])

## Train SVM Model

In [6]:
# Split data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(
    columns, category, random_state=RANDOM_STATE, test_size=TEST_SIZE, shuffle=SHUFFLE)

In [7]:
# Fit SVM model with out train data
svm_w2v.fit(X_train, y_train)

Pipeline(steps=[('word2vec vectorizer',
                 <mean_vectorizer.MeanEmbeddingVectorizer object at 0x7fa83f451640>),
                ('SVM', SVC(C=100, gamma=30))])

In [8]:
# Model Evaluation
svm_w2v.score(X_train, y_train)

0.9847094801223242

In [9]:
# SVM prediction based on test data
prediction = svm_w2v.predict(X_test)
prediction

array([7, 2, 0, 2, 3, 0, 4, 0, 0, 0, 0, 2, 0, 0, 0, 4, 2, 3, 0, 0, 6, 4,
       0, 4, 0, 4, 4, 4, 2, 3, 2, 2, 6, 0, 0, 3, 4, 0, 7, 0, 7, 7, 5, 5,
       4, 7, 5, 3, 7, 3, 6, 3, 6, 5, 2, 3, 0, 2, 0, 8, 4, 0, 0, 2, 0, 0,
       0, 6, 3, 7, 3, 0, 0, 0, 3, 0, 0, 6, 4, 0, 7, 2, 0, 0, 0, 0, 4, 0,
       3, 6, 4, 0, 0, 5, 5, 4, 0, 0, 0, 0, 4, 4, 2, 0, 6, 0, 8, 3, 0, 3,
       4, 0, 0, 0, 6, 2, 4, 3, 8, 4, 0, 2, 0, 0, 2, 5, 3, 7, 5, 3, 4, 0,
       7, 0, 0, 3, 0, 0, 5, 0, 3, 4, 0, 3, 3, 0, 7, 0, 5, 4, 0, 2, 5, 2,
       2, 4, 8, 0, 3, 0, 3, 3, 2, 6, 2, 0, 5, 5, 2, 0, 3, 5, 5, 2, 5, 0,
       7, 7, 3, 0, 0, 0, 6, 0, 0, 0, 0, 0, 3, 0, 0, 3, 2, 0, 3, 3, 2, 0,
       4, 3, 3, 2, 0, 6, 2, 6, 5, 4, 0, 5, 6, 0, 0, 3, 0, 0, 4, 0, 0, 0,
       7, 0, 8, 3, 3, 0, 4, 4, 6, 6, 9, 0, 4, 0, 6, 4, 4, 3, 5, 0, 0, 2,
       4, 7, 4, 7, 0, 3, 3, 0, 0, 0, 3, 2, 5, 3, 3, 0, 4, 0, 0, 4, 0, 3,
       3, 0, 4, 6, 0, 4, 0, 7, 3, 5, 2, 2, 0, 5, 4, 0, 7])

## Evaluation

In [13]:
# View evaluation metrics table
evaluation = model_evaluation_metrics(y_test, prediction, 'Category')
evaluation

,Evaluation Metrics,Category
0,Accuracy,0.498221
1,Precission,0.375095
2,Recall,0.371580
3,F1 Score,0.364762
4,Mean Absolute Error,2.032028
5,Mean Squared Error,10.544484


In [12]:
# Transform columns data to dict to use in MLFlow
transformation = series_pair_to_dict(evaluation, 'Evaluation Metrics', 'Category')
transformation

{'Accuracy': 0.498220640569395,
 'Precission': 0.3750947233403374,
 'Recall': 0.37158024908060977,
 'F1 Score': 0.3647617453399299,
 'Mean Absolute Error': 2.0320284697508897,
 'Mean Squared Error': 10.544483985765124}

In [32]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Category (Word2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'word2vec',
                  "Random state": RANDOM_STATE,
                  "Test size": TEST_SIZE,
                  "Shuffle": SHUFFLE,
                  "Gamma": GAMMA,
                  "C": C}
    log_params(parameters)
    log_metrics(transformation)
